# Setup

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers

Using TensorFlow backend.


# Get Sequences

In [0]:
# Load text
file = open('shakespeare.txt', 'r')
sonnets = file.read()
file.close()

# Strip whitespace and numbers (leaves only words)
tokens = sonnets.split()
all_words = ' '.join(i for i in tokens if not i.isdigit())
all_words = all_words.lower()

# Break into sequences of 40 characters each, starting for each group of 10
N = 5
seq_length = 41
sequences = list()
for i in range(0, len(all_words), N):
  seq = all_words[i : min(i + seq_length, len(all_words))] # Prevent out-of-bounds error
  if len(seq) == seq_length:
    sequences.append(seq)

# Get int Sequences

In [0]:
# Get unique characters and match them to an index
chars = sorted(list(set(all_words)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [0]:
# Create sequences of ints
int_sequences = list()
for seq in sequences:
	int_seq = [mapping[char] for char in seq]
	int_sequences.append(int_seq)

In [0]:
# Separate into X and Y sets (40 characters are input, 1 is output)
int_sequences = np.array(int_sequences)
X, y = int_sequences[:,:-1], int_sequences[:,-1]

# One-hot encode them
class_num = len(mapping)
int_sequences = [to_categorical(x, num_classes = class_num) for x in X]
X = np.array(int_sequences)
y = to_categorical(y, num_classes = class_num)

# Build Model

In [0]:
# Model with LSTM layer of 175 units and softmax activation on output layer
model = tf.keras.Sequential()
model.add(layers.LSTM(175, input_shape=(X.shape[1], X.shape[2]), dropout=0.1))
model.add(layers.Dense(50, activation= 'relu'))
model.add(layers.Dense(class_num, activation='softmax'))

# Define loss function and train
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size = 32, epochs = 100)
print(model.summary())

# Save weights 
saved_weights = [layer.get_weights() for layer in model.layers]

Epoch 1/100
586/586 [==============================] - 44s 74ms/step - loss: 2.8313 - accuracy: 0.2190
Epoch 2/100
586/586 [==============================] - 44s 75ms/step - loss: 2.4192 - accuracy: 0.3082
Epoch 3/100
586/586 [==============================] - 44s 75ms/step - loss: 2.2720 - accuracy: 0.3358
Epoch 4/100
586/586 [==============================] - 44s 75ms/step - loss: 2.1837 - accuracy: 0.3596
Epoch 5/100
586/586 [==============================] - 44s 76ms/step - loss: 2.1121 - accuracy: 0.3744
Epoch 6/100
586/586 [==============================] - 44s 74ms/step - loss: 2.0635 - accuracy: 0.3850
Epoch 7/100
586/586 [==============================] - 44s 74ms/step - loss: 2.0045 - accuracy: 0.4019
Epoch 8/100
586/586 [==============================] - 43s 74ms/step - loss: 1.9612 - accuracy: 0.4133
Epoch 9/100
586/586 [==============================] - 43s 74ms/step - loss: 1.9171 - accuracy: 0.4226
Epoch 10/100
586/586 [==============================] - 43s 74ms/step - l

In [0]:
# Save Model
model_json = model.to_json()
with open('model3.json', 'w') as json_file:
  json_file.write(model_json)
model.save_weights('model3.h5')

In [0]:
# Load Model
json_file = open('model3.json', 'r')
model_json = json_file.read()
json_file.close()
model = tf.keras.models.model_from_json(model_json)
model.load_weights('model3.h5')

# Recompile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
                     metrics=['accuracy'])

saved_weights = [layer.get_weights() for layer in model.layers]

# Generate Poem

In [0]:
# Implement temperature parameter
temp = 0.75

# Model with Lambda layer for temperature
temp_model = tf.keras.Sequential()
temp_model.add(layers.LSTM(175, input_shape=(X.shape[1], X.shape[2]), dropout=0.1))
temp_model.add(layers.Dense(50, activation= 'relu'))
temp_model.add(layers.Lambda(lambda x: x / temp))
temp_model.add(layers.Dense(class_num, activation='softmax'))

# Load saved weights (skipping lambda layer)
temp_model.layers[0].set_weights(saved_weights[0])
temp_model.layers[1].set_weights(saved_weights[1])
temp_model.layers[3].set_weights(saved_weights[2])

In [0]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):

  in_text = seed_text

  for i in range(n_chars):
    encoded = [mapping[char] for char in in_text]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    encoded = to_categorical(encoded, num_classes=len(mapping))

    yhat = model.predict_classes(encoded)

    out_char = ''
    for char, index in mapping.items():
      if index == yhat:
        out_char = char
        break

    in_text += out_char

  return in_text

In [0]:
print(generate_seq(temp_model, mapping, 40, 'shall i compare thee to a summer\'s day?', 14*40))

shall i compare thee to a summer's day? why deft ere but the end, that me nar freme of love, but what wards the should fartered there i am hus were broe, where flager happy fams, and you in whose will in thy self thou save, who are to not free inshorns of their remoring: woo of what it dest some conched as have in liveryino of not reauty being no corrt, me for my love thee, as prove of moly rese, om day, yee in the remory to on these badty though mese feerd, then my own forgh, and 'tiff oo heavi, and therefire wishom me mose foll time and of triies of remove, who i ame seef, touth tris r paig
